# Predict using SageMaker Endpoint

## Model:  MobileNet (v1) SSD  300x300
## Trained For:  CFA Product Images


https://www.tensorflow.org/api_docs/python/tf/io/encode_jpeg


In [ ]:
import io, os, sys
import json
import base64
import time
import numpy as np


from PIL import Image
import IPython.display as display

import tensorflow as tf

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
tf.enable_eager_execution()

In [ ]:

# This is needed since we cloned tensorflow/models under code.
# - if you don't know what this means
#   Look at the notebook TrainModel_Step1_Local
#      in this notebook, you basically set up the project with includes cloning 
#      and compiling the tensorflow/models repo
#   we are using the utilities found in that repo

cwd = os.getcwd()
models = os.path.join(cwd, 'code/models/research/')
slim = os.path.join(cwd, 'code/models/research/slim')
sys.path.append(models)
sys.path.append(slim)

# visualization
from object_detection.utils import ops as utils_ops
from object_detection.utils.visualization_utils import STANDARD_COLORS
from object_detection.utils.visualization_utils import draw_bounding_box_on_image
from object_detection.utils.label_map_util import get_label_map_dict

from code.cfa_utils.example_utils import feature_obj_detect
from code.cfa_utils.example_utils import bytes_feature
from code.cfa_utils.example_utils import parse_function
from code.cfa_utils.example_utils import get_class_names
from code.cfa_utils.example_utils import get_dataset_length
from code.cfa_utils.example_utils import display_detection

## Global Variables

In [ ]:
PROJECT_DIR = os.getcwd()
IMAGE_DIR = os.path.join(PROJECT_DIR, "data/jpeg_images")

# you may have to update your model version number
MODEL_PATH = os.path.join(PROJECT_DIR, "trained_model/export/Servo/1570217480")
LABEL_MAP = os.path.join(PROJECT_DIR, "code/cfa_prod_label_map.pbtxt")

# you can get data using the TrainModel_Step1_Local notebook
# when writing - write to code/tfrecords
# when you pull from S3 and your processing, read from data/tfrecords
TFRECORDS_PATH =  os.path.join(PROJECT_DIR, "tmp")
                                    
SAMPLE_IMAGE = "/home/ec2-user/SageMaker/ssd-dag/data/jpeg_images/20190710_variety_1562781002.jpg"

# NAME - get this from the console
ENDPOINT_NAME = "ep-mobilenet-ssd"  

## Local Model

Local Model was pulled from a successful SageMaker training job (S3 -> local) and extracted.   This verifies the training job:
- created a saved_model.pb
- in export/Servo/

And, we can read the Signature Defs

In [ ]:
print ("Loading saved_model.py from:", MODEL_PATH)
loaded_model = tf.saved_model.load(sess=tf.Session(), 
                                   tags=[tf.saved_model.tag_constants.SERVING], 
                                   export_dir=MODEL_PATH)

In [ ]:
# this model complies to serving framework and can read signature defs
!saved_model_cli show --dir {MODEL_PATH} --tag_set serve 

In [ ]:
# Signatures for:
# - serving_default
# - tensorflow/serving/predict
# appear to be the same
!saved_model_cli show --dir {MODEL_PATH} --tag_set serve --signature_def serving_default

## SageMaker Endpoint
create the endpoint assuming it doesn't already exist.  

you go to the SageMaker console
- Endpoints:  Create
- on Create & Configure
  - name:   ep-mobilenet-ssd  (whatever you want but the global name is in this code - above)
  - endpoint configuration:   use the epc-mobilenet-ssd (this specifies p2.xlarge)
  
This will take 5-10 minutes

THERE ARE MORE NOTES in the TrainModel_Step3_TrainingJob.  Creating an endpoint configuration requires knowing the inference code image (Docker?) - and I haven't figured out how to get that from the training job.

if it fails...
- retrain a model just to make sure you have a good one
- recreate the endpoint config - this seems to be the most important artifact
  

In [ ]:
import sagemaker
from sagemaker.tensorflow.model import TensorFlowModel
from sagemaker.predictor import json_serializer, json_deserializer

sagemaker_session = sagemaker.Session()
predictor=sagemaker.tensorflow.model.TensorFlowPredictor(ENDPOINT_NAME, sagemaker_session)

print (type(predictor))

## Single Image Prediction

In [ ]:
# Eager execution IS turned on
image_tensor = tf.io.read_file(SAMPLE_IMAGE)   # EagerTensor
image_bytes = image_tensor.numpy()              # bytes
print (type(image_bytes))

# create a feature
feature = {}
feature['image/encoded'] = bytes_feature(image_bytes)
print (type(feature))

# create a tf.train.Features
# - because you'll need this when you create tf.train.Example
features = tf.train.Features(feature=feature)
print (type(features))

# now create the tf.train.Example
ex = tf.train.Example(features=features)
print (type(ex))

# serialize the Example to a string
ex_str = ex.SerializePartialToString()
print (type(ex_str))

In [ ]:
# create the request
# specify the signature name
# instances must be a list
d = {'signature_name': 'serving_default', 'instances': [{'b64': base64.standard_b64encode(ex_str).decode('ascii')}]}

In [ ]:
# Call the model
model_response = predictor.predict(d)

In [ ]:
print (type(model_response))
print ("model response keys:", model_response.keys())

# predictios is a list
predictions = model_response['predictions']
predictions_count = len(predictions)
print ("predictions returned", predictions_count)
print ("predictions keys:", predictions[0].keys())

## Examine the model response

#### The response structure was documented in the model query (cli) you ran above

- you see raw_* responses
- you see post processed responses - we are using only post processed response data;  note that this is controlled by the pipeline config file

In [ ]:
# there are 12 object classes defined in code/cfa_prod_label_map.pbtxt
print ("number of predictions - per image:", len(predictions[0]['raw_detection_scores']))
print ("number of classes per prediction:", len(predictions[0]['raw_detection_scores'][0]))

# raw detection scores
# - this is probably related to anchor boxes
#   so 1917 is based on the dimensions and the model defsl
raw_scores = np.asarray(predictions[0]['raw_detection_scores'], dtype=np.float32)
print (type(raw_scores), raw_scores.shape)

# detection scores
# in the config:  max_number_of_boxes: 100
# - so, you have 100 top scores
detect_scores = np.asarray(predictions[0]['detection_scores'], dtype=np.float32) 
print ("detect scores:", type(detect_scores), detect_scores.shape, detect_scores[:10])

# detection classes
detect_classes = np.asarray(predictions[0]['detection_classes'], dtype=np.int8)
#np.reshape(detect_classes, (?,))
print ("detect class ids:", type(detect_classes), detect_classes.shape, detect_classes[:10])

# detection boxes
# - note they are normalized
detect_boxes = np.asarray(predictions[0]['detection_boxes'], dtype=np.float32) 
print ("bounding boxes:", type(detect_boxes), detect_boxes.shape, '\n', detect_boxes[:10])

In [ ]:
# get the class names from the label map
class_names = get_class_names(LABEL_MAP)

display_detection(image_tensor, class_names, 0.5, predictions[0])


## Process from TFRecord file

parse_single_example  
https://github.com/tensorflow/tensorflow/blob/r2.0/tensorflow/python/ops/parsing_ops.py#L1025-L1072

In [ ]:


tfrecord_file_list_input = [os.path.join(TFRECORDS_PATH, 'train/train.tfrecord'),
                            os.path.join(TFRECORDS_PATH, 'val/val.tfrecord'),
                            os.path.join(TFRECORDS_PATH, 'test/test.tfrecord')]
print (tfrecord_file_list_input)
raw_dataset = tf.data.TFRecordDataset(tfrecord_file_list_input)
raw_dataset.shuffle(buffer_size = 5000)
print (type(raw_dataset))
# brute force utility to get record count
# gonna take 10-20 sec
dataset_length = get_dataset_length(raw_dataset)
print (dataset_length)

In [ ]:
parsed_dataset = raw_dataset.map(parse_function)

In [ ]:
display_nth = 100
# get the class names from the label map
class_names = get_class_names(LABEL_MAP)



for n,i in enumerate(parsed_dataset.take(1000)):
    # timing gate
    start_time = time.time()
    # print ("start time: {:.4f}".format(start_time))
    
    image_bytes = i['image/encoded'].numpy()  # bytes
    # create a feature
    feature = {}
    feature['image/encoded'] = bytes_feature(image_bytes)
    # create a tf.train.Features
    # - because you'll need this when you create tf.train.Example
    features = tf.train.Features(feature=feature)
    # now create the tf.train.Example
    ex = tf.train.Example(features=features)
    # serialize the Example to a string
    ex_str = ex.SerializePartialToString()

    d = {'signature_name': 'serving_default', 'instances': [{'b64': base64.standard_b64encode(ex_str).decode('ascii')}]}
    
    model_response = predictor.predict(d)
    predictions = model_response['predictions']
    if n % display_nth == 0:
        display_detection(i['image/encoded'], class_names, 0.5, predictions[0])
    
    # show the time
    finish_time = time.time()
    minutes = (finish_time - start_time) / 60
    print(n, "time spent: {:.4f}".format(finish_time - start_time), " in minutes: {:.4f}".format(minutes))
    
    
    
